In [3]:
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import StructuredTool
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# def plus(a, b):
#     return a + b


# # structured input ReAct(Reasoning and Acting) -> not related to reactjs lol
# # * how an agent works like a loop and langchain does pass the context.
# # langchane behaves like a bridge -> sometimes it can be broken
# # -> better option: OpenAI function calling -> force the output
# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     tools=[
#         StructuredTool.from_function(
#             func=plus,
#             name="Sum Calculator",
#             description="A calculator that adds two numerical values and returns their sum. Both arguments must be numbers.",
#         )
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# # llm.invoke(prompt)
# agent.run(prompt)
# # agent.invoke({"input": prompt})

In [4]:
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import Tool
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# def plus(inputs):
#     a, b = inputs.split(",")
#     return float(a) + float(b)


# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     # Deprecated since version 0.1.0: LangChain agents will continue to be supported,
#     # but it is recommended for new use cases to be built with LangGraph.
#     # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     # It works well now without setting True
#     handle_parsing_errors=True,
#     tools=[
#         Tool.from_function(
#             func=plus,
#             name="Sum Calculator",
#             description="A calculator that adds two numerical values and returns their sum. Use this tool by sending a pair of numbers separated by a comma. \n Example:1,2",
#         )
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# # llm.invoke(prompt)
# agent.run(prompt)
# # agent.invoke({"input": prompt})

In [5]:
# from typing import Type
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import BaseTool
# from pydantic import BaseModel, Field
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# # define args types by using pydantic
# class CalculatorToolArgsSchema(BaseModel):
#     # if set int, langchain automatically rounds values
#     a: float = Field(discription="The first number")
#     b: float = Field(discription="The second number")


# class CalculatorTool(BaseTool):

#     name = "CalculatorTool"
#     description = """
#     Use this to perform sums of two numbers.
#     the first and second arguments must be numbers
#     only receive two numbers.
#     """
#     args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

#     # now the data structure is simple, but if prepare for bit data structures,
#     # separating function is better than just repeat a + b in return
#     def _run(self, a, b):
#         return a + b


# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     # Deprecated since version 0.1.0: LangChain agents will continue to be supported,
#     # but it is recommended for new use cases to be built with LangGraph.
#     # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html
#     agent=AgentType.OPENAI_FUNCTIONS,
#     # It works well now without setting True
#     handle_parsing_errors=True,
#     tools=[
#         CalculatorTool(),
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# agent.invoke(prompt)

In [6]:
# from typing import Type
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import BaseTool
# from pydantic import BaseModel, Field
# from langchain.agents import initialize_agent, AgentType
# from langchain.utilities import DuckDuckGoSearchAPIWrapper
# from langchain.tools import DuckDuckGoSearchResults

# llm = ChatOpenAI(temperature=0.1)


# class StockMarketSymbolSearchToolArgsSchema(BaseModel):
#     query: str = Field(description="The query you will search for")


# class StockMarketSymbolSearchTool(BaseTool):
#     name = "StockMarketSymbolSearchTool"
#     description = """
#     Use this tool to find the stock market symbol of a company.
#     It takes a query as an argument.
#     Example query: Stock Market Symbol for Apple Company
# """
#     args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
#         StockMarketSymbolSearchToolArgsSchema
#     )

#     def _run(self, query):
#         wrapper = DuckDuckGoSearchAPIWrapper(
#             region="wt-wt",  # Default global region
#             time="y",  # Search period (y: 1 year)
#             max_results=5,  # Maximum number of results
#         )
#         search = DuckDuckGoSearchResults(
#             api_wrapper=wrapper,
#             backend="html",
#         )
#         return search.run(query)


# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     agent=AgentType.OPENAI_FUNCTIONS,
#     handle_parsing_errors=True,
#     tools=[
#         StockMarketSymbolSearchTool(),
#     ],
# )

# prompt = "Give me information of Cloudflare's stock and help me help me analyze if it's a potential good investment. And also tell me what stock symbol does the stock have."


# agent.invoke(prompt)

In [7]:
# import os
# from typing import Type
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import BaseTool
# from pydantic import BaseModel, Field
# from langchain.agents import initialize_agent, AgentType
# from langchain.utilities import DuckDuckGoSearchAPIWrapper
# from langchain.tools import DuckDuckGoSearchResults
# import requests

# llm = ChatOpenAI(
#     temperature=0.1,
#     model="gpt-4.1-nano",
# )

# alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


# class StockMarketSymbolSearchToolArgsSchema(BaseModel):
#     query: str = Field(description="The query you will search for")


# class StockMarketSymbolSearchTool(BaseTool):
#     name = "StockMarketSymbolSearchTool"
#     description = """
#     Use this tool to find the stock market symbol of a company.
#     It takes a query as an argument.
#     Example query: Stock Market Symbol for Apple Company
#     """
#     args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
#         StockMarketSymbolSearchToolArgsSchema
#     )

#     def _run(self, query):
#         wrapper = DuckDuckGoSearchAPIWrapper(
#             region="wt-wt",  # Default global region
#             time="y",  # Search period (y: 1 year)
#             max_results=5,  # Maximum number of results
#         )
#         search = DuckDuckGoSearchResults(
#             api_wrapper=wrapper,
#             backend="html",
#         )
#         return search.run(query)


# class CompanyArgsSchema(BaseModel):

#     symbol: str = Field(description="Stock symbol of the company.Example: AAPL, TSLA")


# class CompanyOverviewTool(BaseTool):
#     name = "CompanyOverview"
#     description = """
#     Use this to get an overview of the financials of the company.
#     You should enter a stock symbol.
#     """
#     args_schema: Type[CompanyArgsSchema] = CompanyArgsSchema

#     def _run(self, symbol):
#         url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
#         r = requests.get(url)
#         return r.json()


# class CompanyIncomeStatementTool(BaseTool):
#     name = "CompanyIncomeStatement"
#     description = """
#     Use this to get an income statement of a company.
#     You should enter a stock symbol.
#     """
#     args_schema: Type[CompanyArgsSchema] = CompanyArgsSchema

#     def _run(self, symbol):
#         url = f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
#         r = requests.get(url)
#         return r.json()
#         # return r.json()["annualReports"]


# class CompanyStockPerformanceTool(BaseTool):
#     name = "CompanyStockPerformance"
#     description = """
#     Use this to get a weekly performance of a company stock.
#     You should enter a stock symbol.
#     """
#     args_schema: Type[CompanyArgsSchema] = CompanyArgsSchema

#     def _run(self, symbol):
#         url = f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
#         r = requests.get(url)
#         return r.json()


# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     agent=AgentType.OPENAI_FUNCTIONS,
#     handle_parsing_errors=True,
#     tools=[
#         StockMarketSymbolSearchTool(),
#         CompanyOverviewTool(),
#         CompanyIncomeStatementTool(),
#         CompanyStockPerformanceTool(),
#     ],
# )

# prompt = "Give me financial information on Cloudflare's stock. Please use both overview, income statements, and stock performance data to help me analyze if it's a good potential investment."


# agent.invoke(prompt)

In [8]:
# d = {
#     "WeeklyTimes": {
#         "x1": 1,
#         "x2": 2,
#         "x3": 3,
#     }
# }

# d["WeeklyTimes"]

# list(d["WeeklyTimes"].items())[:2]

In [ ]:
from json import tool
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
)

db = SQLDatabase.from_uri("sqlite:///../movies.sqlite")

toolkit = SQLDatabaseToolkit(
    db=db,
    llm=llm,
)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
)

# * first try
# agent.invoke(
#     {
#         "input": "Give me the 5 directors that have the highst grossing films.",
#     },
# )

# * second try
# agent.invoke(
#     {
#         "input": "Give me the movies that have the highest votes but the lowest budgets, and give me the names of the directors. Also, include their gross revenues and budgets",
#     },
# )

# get information of toolkit
toolkit.get_tools()

In [ ]:
from langchain.tools import toolkit


toolkit.get_tools()